In [1]:
import numpy as np
import pandas as pd

from random import randint
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

from skimage.transform import resize

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input,Dense, AveragePooling2D,Conv2D, Conv2DTranspose,Flatten, MaxPooling2D, concatenate, Dropout,BatchNormalization,LeakyReLU,Activation,Add
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from tqdm import tqdm_notebook
from keras import backend as K
from keras import optimizers
import time

Using TensorFlow backend.


In [20]:
#model = load_model('/home/wenjie/Desktop/earImageDataset/keras-frcnn-master.2/saved_model/direct_92_val.model')
model = load_model('./saved_model/Triplet_trainable.model')

In [3]:
data_df = pd.read_csv('./easy_more.csv',index_col=0)
data = data_df.sample(frac=1).reset_index(drop=True)

In [4]:
img_path = '../resized_cropped_detection'
imgs = os.listdir(img_path)

In [5]:
img_dic = {}
for img in imgs:
    img_dic[img] = np.array(load_img(img_path+'/'+img))/255
    print(img)
print('Read_Done')

140_d.jpg
016_dt.jpg
102_dt.jpg
054_t.jpg
171_d.jpg
164_.jpg
091_.jpg
048_t.jpg
020_dt.jpg
180_dt.jpg
177_dt.jpg
187_t.jpg
001_.jpg
064_dt.jpg
190_dt.jpg
118_dt.jpg
186_dt.jpg
064_.jpg
188_t.jpg
090_t.jpg
098_.jpg
179_t.jpg
033_d.jpg
052_d.jpg
048_d.jpg
160_d.jpg
066_dt.jpg
096_d.jpg
088_.jpg
152_d.jpg
136_t.jpg
143_.jpg
119_d.jpg
013_.jpg
006_.jpg
192_d.jpg
062_t.jpg
179_dt.jpg
193_t.jpg
172_.jpg
095_dt.jpg
019_.jpg
003_d.jpg
020_t.jpg
012_d.jpg
186_d.jpg
175_dt.jpg
184_t.jpg
022_.jpg
147_t.jpg
140_dt.jpg
126_.jpg
027_d.jpg
029_d.jpg
075_d.jpg
022_dt.jpg
130_.jpg
162_d.jpg
010_d.jpg
104_d.jpg
144_d.jpg
001_dt.jpg
004_t.jpg
040_dt.jpg
114_.jpg
151_d.jpg
094_d.jpg
072_dt.jpg
165_d.jpg
117_t.jpg
097_dt.jpg
169_dt.jpg
173_t.jpg
086_t.jpg
173_.jpg
072_d.jpg
120_dt.jpg
181_dt.jpg
031_.jpg
153_dt.jpg
080_t.jpg
128_t.jpg
070_d.jpg
094_t.jpg
123_.jpg
195_.jpg
063_.jpg
074_t.jpg
163_.jpg
056_t.jpg
060_d.jpg
122_dt.jpg
080_dt.jpg
042_dt.jpg
094_.jpg
071_dt.jpg
014_.jpg
141_.jpg
115_.jpg
178_dt.j

In [6]:
train_data = data.iloc[0:3000,:]

In [7]:
x_1 = np.array([img_dic[k] for k in train_data['Img1']])
x_2 = np.array([img_dic[k] for k in train_data['Img2']])
#target = np.array(train_data['Target'])
target = [[0,0] for i in range(train_data.shape[0])]
s = np.array(train_data['Target'])
for i in range(train_data.shape[0]):
    target[i][1-s[i]] = 1
target = np.array(target)

In [8]:
target[0:10]

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1]])

In [9]:
x_1[0].shape

(224, 224, 3)

In [21]:
pre = model.predict([x_1,x_2])

In [22]:
pre[0:10]

array([[2.6481659e-06, 9.9999738e-01],
       [5.4370514e-07, 9.9999940e-01],
       [2.5007993e-04, 9.9974996e-01],
       [1.9452929e-03, 9.9805474e-01],
       [6.3419271e-01, 3.6580732e-01],
       [1.2431413e-08, 1.0000000e+00],
       [9.3483633e-01, 6.5163672e-02],
       [2.8649114e-02, 9.7135091e-01],
       [2.0790262e-02, 9.7920978e-01],
       [1.6403462e-04, 9.9983597e-01]], dtype=float32)

In [23]:
#pre_ind = [1-(i[0] > i[1]) for i in pre] 
pre_ind = np.argmax(pre,axis = 1) 

In [24]:
pre_ind

array([1, 1, 1, ..., 0, 1, 1])

In [25]:
target = np.array(train_data['Target'])

In [26]:
target = np.array(target)

In [27]:
acc = sum(abs(target - pre_ind))/target.shape[0]

In [28]:
acc

0.7873333333333333